In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

newnames_reco = {"aice_body-sharp":"DLR", 
                 "first_body":"MBIR",                  
                 "fbp_fc08":"FBP", 
                 "aidr3d_fc08":"HIR", 
                 "asir_standard":"HIR", 
                 "fbp_standard":"FBP",
                 "qr40f-q3_0.4":"Qr40f-Q3_0.4",
                 "qr40f-q3_0.6":"Qr40f-Q3_0.6",
                 "qr40f-q3_1":"Qr40f-Q3_1",
                 "br40f-q3_0.4":"Br40f-Q3_0.4",
                 "br40f-q3_0.6":"Br40f-Q3_0.6",
                 "br40f-q3_1":"Br40f-Q3_1"}

newnames_ct = {
    "CT4": "<b>Vendor 1",
    "GE": "<b>Vendor 2",
    "photon":"<b>Vendor 3",
}

newnames_network = {
    "3d_fullres_LiTS_151":"<b>3D",
    "3d_fullres_LiTS_151_res":"<b>3Dres",
    "3d_fullres_LiTS+phantom_254":"<b>3D improved",
    "3d_fullres_LiTS+phantom_254_res":"<b>3Dres improved"
}
size_filter = 4/3 * 3.14159265359 * (5/2)**3   # in cubic millimeters, 5mm sphere

dice_detection_threshold = 0.0
output_dir = '/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/plots/3) deep learning performance eval/lesion_groups'

In [ ]:
df_lesion_meta = pd.read_csv('results_csv/results_lesion_paper.csv')
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"volume_pred"] = None
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"dice"] = None
df_lesion_meta = df_lesion_meta[(df_lesion_meta["volume_pred"] > size_filter) | (df_lesion_meta["volume_pred"].isna()) ]

df_lesion_meta["detected"] = df_lesion_meta["dice"] > dice_detection_threshold 
filter_lesion_props = (df_lesion_meta["C in HU"] < -10) | (df_lesion_meta["C in HU"].isna())
df_lesion_meta = df_lesion_meta[filter_lesion_props]
df_lesion_meta = df_lesion_meta[df_lesion_meta["kernel"] != "FC13"]

df_lesion_meta["ct"] = df_lesion_meta["ct"].map(newnames_ct)
df_lesion_meta = df_lesion_meta[df_lesion_meta["date"].isin([230915, 231205, 240306])]

In [ ]:
df_lesion_meta["lesion_prop"] = df_lesion_meta.apply(lambda x: f"{str(2*int(x['R in mm'])).zfill(2)}mm/{int(x['C in HU'])}HU" if not np.isnan(x['R in mm']) else None , axis=1)

df_lesion_meta["reco+kernel"] = df_lesion_meta["reco"].str.lower() + "_" + df_lesion_meta["kernel"].str.lower()
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco+kernel"].map(newnames_reco)
df_lesion_meta["network"] = df_lesion_meta["network"].map(newnames_network)


In [ ]:
groupby_lesion_prop = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU"]
groupby_lesion_prop_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "rep"]
groupby_lesion_all = ["network", "CTDIvol", "reco+kernel", "ct"]
groupby_lesion_prop_phantom = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "phantom"]
groupby_lesion_prop_phantom_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "phantom", "rep"]


In [ ]:
df_lesion_meta_recall_phantom_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()]
df_lesion_meta_recall_phantom_rep = df_lesion_meta_recall_phantom_rep.groupby(groupby_lesion_prop_phantom_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_phantom_rep.columns = groupby_lesion_prop_phantom_rep + ["recall", "num_detected", "num_total"]
df_lesion_meta_precision_phantom_rep = df_lesion_meta.groupby(groupby_lesion_prop_phantom_rep).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"]}).reset_index()
df_lesion_meta_precision_phantom_rep.columns = groupby_lesion_prop_phantom_rep + ["precision", "num_detected", "num_total", "dice_mean", "dice_std"]
lesion_seg_summary_prop_phantom_rep = df_lesion_meta_recall_phantom_rep.merge(df_lesion_meta_precision_phantom_rep, on=groupby_lesion_prop_phantom_rep, how="left")
lesion_seg_summary_prop_phantom_rep["F1-score"] = 2 * (lesion_seg_summary_prop_phantom_rep["precision"] * lesion_seg_summary_prop_phantom_rep["recall"]) / (lesion_seg_summary_prop_phantom_rep["precision"] + lesion_seg_summary_prop_phantom_rep["recall"])
lesion_seg_summary_prop_phantom_rep["recall_min_rep"] = lesion_seg_summary_prop_phantom_rep.groupby(groupby_lesion_prop_phantom)["recall"].transform("min")

In [ ]:
df_lesion_meta_recall_prop_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()]
df_lesion_meta_recall_prop_rep = df_lesion_meta_recall_prop_rep.groupby(groupby_lesion_prop_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_prop_rep.columns = groupby_lesion_prop_rep + ["recall", "num_detected", "num_total"]

# min recall reached per lesion type and dose across all repetitions 
df_lesion_meta_recall_prop_rep_min = df_lesion_meta_recall_prop_rep.groupby(groupby_lesion_prop)["recall"].transform("min")
df_lesion_meta_recall_prop_rep["recall_min_rep"] = df_lesion_meta_recall_prop_rep_min


numb_stable = 5
# check if min recall reached in all repetitions is stable across doses
# check for how many consecutive doses the min recall is reached
df_lesion_meta_recall_prop_rep = df_lesion_meta_recall_prop_rep.sort_values(["network", "ct", "reco+kernel", "lesion_prop", "CTDIvol" ,"rep"])
for i in range(1, numb_stable):
    df_lesion_meta_recall_prop_rep[f"recall_min_rep_{i}"] = df_lesion_meta_recall_prop_rep["recall_min_rep"].shift(-i*5)
    df_lesion_meta_recall_prop_rep[f"recall_min_rep_{i}"] = df_lesion_meta_recall_prop_rep[f"recall_min_rep_{i}"].fillna(0)
    df_lesion_meta_recall_prop_rep[f"recall_min_stable_{i}"] = df_lesion_meta_recall_prop_rep["recall_min_rep"] <= df_lesion_meta_recall_prop_rep[f"recall_min_rep_{i}"]


df_lesion_meta_recall_prop_rep["recall_min_stable"] = df_lesion_meta_recall_prop_rep[[f"recall_min_stable_{i}" for i in range(1, numb_stable)]].all(axis=1)


    
df_lesion_meta_recall_prop_rep_stable = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["recall_min_stable"]]



In [ ]:
# max recall per lesion prop reached in all repetitions
df_lesion_meta_recall_prop_rep_max = df_lesion_meta_recall_prop_rep_stable.\
    groupby(["network", "reco+kernel", "ct", "lesion_prop"])["recall_min_rep"].transform("max")
dose_indicies = (df_lesion_meta_recall_prop_rep_max == df_lesion_meta_recall_prop_rep_stable["recall_min_rep"])
df_lesion_meta_recall_prop_rep_max_recall = df_lesion_meta_recall_prop_rep_stable[dose_indicies]

In [ ]:
df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall.\
    groupby(["network", "reco+kernel", "ct", "lesion_prop"])["CTDIvol"].transform("min")
dose_indicies = df_lesion_meta_recall_prop_rep_max_recall_min_dose == df_lesion_meta_recall_prop_rep_max_recall["CTDIvol"]
df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall[dose_indicies]

In [ ]:
df_lesion_meta_recall_prop = df_lesion_meta_recall_prop_rep.groupby(groupby_lesion_prop).agg({"recall": ["mean", "std"], 
                                                                                              "num_detected": ["mean", "std"], 
                                                                                              "num_total": ["mean", "std"],
                                                                                              "recall_min_rep": ["mean", "std"]}).reset_index()

df_lesion_meta_recall_prop.columns = groupby_lesion_prop + ["recall_mean", "recall_std", 
                                                            "num_detected_mean", "num_detected_std", 
                                                            "num_total_mean", "num_total_std", 
                                                            "recall_min_rep_mean", "recall_min_rep_std"]

In [ ]:
# get percent detected at different doses
import numpy as np

# recall or "recall_min_rep_mean"
metric = "recall_min_rep"
# I calculate the relative dose to the max dose reached
# II calculate the percent detected at the relative doses 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 using interploation
groupby=["lesion_prop","reco+kernel", "network", "ct","rep"]
df_lesion_meta_recall_prop_rep = df_lesion_meta_recall_prop_rep\
    .sort_values(by=groupby)

perc_det_summary_thresholds = {
    "lesion_prop": [],
    "rep":[],
    "reco+kernel": [],
    "network": [],
    "ct": [],
    "CTDIvol": [],
    "relative_dose": [],
    "recall": [],
    "C in HU": [],
    "R in mm": [],
    "interpolated": []
}

for lesion_prop in df_lesion_meta_recall_prop["lesion_prop"].unique():
    for reco_kernel in df_lesion_meta_recall_prop["reco+kernel"].unique():
        for network in df_lesion_meta_recall_prop["network"].unique():
            for ct in df_lesion_meta_recall_prop["ct"].unique():
                for rep in range(1, 6):
                    if ct == "<b>Vendor 2" and (reco_kernel == "DLR" or reco_kernel == "MBIR"):
                        continue
                    if ct == "<b>Vendor 3" and (reco_kernel in ["FBP", "HIR", "MBIR", "DLR"] or rep != 1 or network == "<b>3D improved"):
                        continue
                    if ct in ["<b>Vendor 1", "<b>Vendor 2" ] and reco_kernel not in ["FBP", "HIR", "MBIR", "DLR"]:
                        continue
                    min_dose_max_reached_temp = df_lesion_meta_recall_prop_rep_max_recall_min_dose[(df_lesion_meta_recall_prop_rep_max_recall_min_dose["lesion_prop"] == lesion_prop) & 
                                                                                                (df_lesion_meta_recall_prop_rep_max_recall_min_dose["reco+kernel"] == reco_kernel) & 
                                                                                                (df_lesion_meta_recall_prop_rep_max_recall_min_dose["network"] == network) & 
                                                                                                (df_lesion_meta_recall_prop_rep_max_recall_min_dose["ct"] == ct) & 
                                                                                                (df_lesion_meta_recall_prop_rep_max_recall_min_dose["rep"] == rep)]
                    if len(min_dose_max_reached_temp) == 0:
                        print(lesion_prop, reco_kernel, network, ct, rep)
                    
                    dose_max_reached = min_dose_max_reached_temp["CTDIvol"].values[0]
                        
                    target_ind = (df_lesion_meta_recall_prop_rep["lesion_prop"] == lesion_prop) & \
                                (df_lesion_meta_recall_prop_rep["reco+kernel"] == reco_kernel) & \
                                (df_lesion_meta_recall_prop_rep["network"] == network) & \
                                (df_lesion_meta_recall_prop_rep["ct"] == ct) & \
                                (df_lesion_meta_recall_prop_rep["rep"] == rep)

                    CTDIvol = df_lesion_meta_recall_prop_rep["CTDIvol"][target_ind]
                    df_lesion_meta_recall_prop_rep.loc[target_ind, "relative_dose"] = CTDIvol/dose_max_reached


                    for relative_dose_temp in np.arange(0, 1.05, 0.1):
                        relative_dose_temp = np.round(relative_dose_temp, 2)
                        # find the two closest points
                        relative_dose_known = df_lesion_meta_recall_prop_rep["relative_dose"][target_ind]
                        if relative_dose_temp in relative_dose_known.values:
                            index = relative_dose_known[relative_dose_known == relative_dose_temp].index[0]
                            perc_detected_temp = df_lesion_meta_recall_prop_rep[metric][index]
                            interpolated = False
                        else:
                            interpolated = True
                            dose_diff = relative_dose_known - relative_dose_temp
                            dose_diff_neg = dose_diff[dose_diff < 0]                            
                            dose_diff_pos = dose_diff[dose_diff > 0]
                            dose_diff_neg = dose_diff_neg.sort_values(ascending=False)
                            dose_diff_pos = dose_diff_pos.sort_values()
                            if dose_diff_neg.empty:
                                point_1 = (0,0)
                            else:    
                                dose_closest_1_row_index = dose_diff_neg.index[0]
                                point_1 = (df_lesion_meta_recall_prop_rep["relative_dose"][dose_closest_1_row_index], 
                                        df_lesion_meta_recall_prop_rep[metric][dose_closest_1_row_index])

                            if dose_diff_pos.empty: 
                                point_2 = (1,1)
                            else:
                                dose_closest_2_row_index = dose_diff_pos.index[0]
                                point_2 = (df_lesion_meta_recall_prop_rep["relative_dose"][dose_closest_2_row_index], 
                                        df_lesion_meta_recall_prop_rep[metric][dose_closest_2_row_index])
                            #print(point_1, point_2)
                            #if point_1[0] > point_2[0]:
                            #    point_1, point_2 = point_2, point_1
                            # interpolate
                            if point_2[0] == point_1[0]:
                                perc_detected_temp = point_1[1]
                            else:
                                m = (point_2[1] - point_1[1])/(point_2[0] - point_1[0])
                                n = point_1[1] - m*point_1[0]
                                perc_detected_temp = m*relative_dose_temp + n

                                #print(lesion_prop, reco_kernel, network, ct, relative_dose_temp, perc_detected_temp)
                        
                        
                        perc_det_summary_thresholds["lesion_prop"].append(lesion_prop)
                        perc_det_summary_thresholds["rep"].append(rep)
                        perc_det_summary_thresholds["C in HU"].append(df_lesion_meta_recall_prop_rep["C in HU"][target_ind].values[0])
                        perc_det_summary_thresholds["R in mm"].append(df_lesion_meta_recall_prop_rep["R in mm"][target_ind].values[0])
                        perc_det_summary_thresholds["reco+kernel"].append(reco_kernel)
                        perc_det_summary_thresholds["network"].append(network)
                        perc_det_summary_thresholds["ct"].append(ct)
                        perc_det_summary_thresholds["CTDIvol"].append(dose_max_reached*relative_dose_temp)
                        perc_det_summary_thresholds["relative_dose"].append(relative_dose_temp)
                        perc_det_summary_thresholds["recall"].append(perc_detected_temp)
                        perc_det_summary_thresholds["interpolated"].append(interpolated)

perc_det_summary_thresholds = pd.DataFrame(perc_det_summary_thresholds)

In [ ]:
# get percent detected at different doses
import numpy as np

# recall or "recall_min_rep_mean"
metric = "recall_min_rep"
# I calculate the relative dose to the max dose reached
# II calculate the percent detected at the relative doses 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9 using interploation
groupby=["lesion_prop","reco+kernel", "network", "ct","rep"]
df_lesion_meta_recall_prop_rep = df_lesion_meta_recall_prop_rep\
    .sort_values(by=groupby)

reference_dose = 3

perc_det_summary_thresholds_fixed_reference = {
    "lesion_prop": [],
    "rep":[],
    "reco+kernel": [],
    "network": [],
    "ct": [],
    "CTDIvol": [],
    "relative_dose": [],
    "recall": [],
    "C in HU": [],
    "R in mm": [],
    "interpolated": []
}

for lesion_prop in df_lesion_meta_recall_prop["lesion_prop"].unique():
    for reco_kernel in df_lesion_meta_recall_prop["reco+kernel"].unique():
        for network in df_lesion_meta_recall_prop["network"].unique():
            for ct in df_lesion_meta_recall_prop["ct"].unique():
                for rep in range(1, 6):
                    if ct == "<b>Vendor 2" and (reco_kernel == "DLR" or reco_kernel == "MBIR"):
                        continue
                    if ct == "<b>Vendor 3" and (reco_kernel in ["FBP", "HIR", "MBIR", "DLR"] or rep != 1 or network == "<b>3D improved"):
                        continue
                    if ct in ["<b>Vendor 1", "<b>Vendor 2" ] and reco_kernel not in ["FBP", "HIR", "MBIR", "DLR"]:
                        continue
                    
                    
                        
                    target_ind = (df_lesion_meta_recall_prop_rep["lesion_prop"] == lesion_prop) & \
                                (df_lesion_meta_recall_prop_rep["reco+kernel"] == reco_kernel) & \
                                (df_lesion_meta_recall_prop_rep["network"] == network) & \
                                (df_lesion_meta_recall_prop_rep["ct"] == ct) & \
                                (df_lesion_meta_recall_prop_rep["rep"] == rep)

                    CTDIvol = df_lesion_meta_recall_prop_rep["CTDIvol"][target_ind]
                    df_lesion_meta_recall_prop_rep.loc[target_ind, "relative_dose"] = CTDIvol/reference_dose
                    

                    for relative_dose_temp in np.arange(0, 1.05, 0.1):
                        relative_dose_temp = np.round(relative_dose_temp, 2)
                        # find the two closest points
                        relative_dose_known = df_lesion_meta_recall_prop_rep["relative_dose"][target_ind]
                        if relative_dose_temp in relative_dose_known.values:
                            index = relative_dose_known[relative_dose_known == relative_dose_temp].index[0]
                            perc_detected_temp = df_lesion_meta_recall_prop_rep[metric][index]
                            interpolated = False
                        else:
                            interpolated = True
                            dose_diff = relative_dose_known - relative_dose_temp
                            dose_diff_neg = dose_diff[dose_diff < 0]                            
                            dose_diff_pos = dose_diff[dose_diff > 0]
                            dose_diff_neg = dose_diff_neg.sort_values(ascending=False)
                            dose_diff_pos = dose_diff_pos.sort_values()
                            if dose_diff_neg.empty:
                                point_1 = (0,0)
                            else:    
                                dose_closest_1_row_index = dose_diff_neg.index[0]
                                point_1 = (df_lesion_meta_recall_prop_rep["relative_dose"][dose_closest_1_row_index], 
                                        df_lesion_meta_recall_prop_rep[metric][dose_closest_1_row_index])

                            if dose_diff_pos.empty: 
                                point_2 = (1,1)
                            else:
                                dose_closest_2_row_index = dose_diff_pos.index[0]
                                point_2 = (df_lesion_meta_recall_prop_rep["relative_dose"][dose_closest_2_row_index], 
                                        df_lesion_meta_recall_prop_rep[metric][dose_closest_2_row_index])
                            #print(point_1, point_2)
                            #if point_1[0] > point_2[0]:
                            #    point_1, point_2 = point_2, point_1
                            # interpolate
                            if point_2[0] == point_1[0]:
                                perc_detected_temp = point_1[1]
                            else:
                                m = (point_2[1] - point_1[1])/(point_2[0] - point_1[0])
                                n = point_1[1] - m*point_1[0]
                                perc_detected_temp = m*relative_dose_temp + n

                                #print(lesion_prop, reco_kernel, network, ct, relative_dose_temp, perc_detected_temp)
                        # if reco_kernel == "DLR" and relative_dose_temp == 1 and network == "<b>3D" and lesion_prop == "12mm/-40HU":
                        #     print(point_1, point_2)
                        #     print(dose_diff_neg)
                        #     print(lesion_prop, reco_kernel, network, ct, relative_dose_temp, perc_detected_temp)
                        
                        perc_det_summary_thresholds_fixed_reference["lesion_prop"].append(lesion_prop)
                        perc_det_summary_thresholds_fixed_reference["rep"].append(rep)
                        perc_det_summary_thresholds_fixed_reference["C in HU"].append(df_lesion_meta_recall_prop_rep["C in HU"][target_ind].values[0])
                        perc_det_summary_thresholds_fixed_reference["R in mm"].append(df_lesion_meta_recall_prop_rep["R in mm"][target_ind].values[0])
                        perc_det_summary_thresholds_fixed_reference["reco+kernel"].append(reco_kernel)
                        perc_det_summary_thresholds_fixed_reference["network"].append(network)
                        perc_det_summary_thresholds_fixed_reference["ct"].append(ct)
                        perc_det_summary_thresholds_fixed_reference["CTDIvol"].append(reference_dose*relative_dose_temp)
                        perc_det_summary_thresholds_fixed_reference["relative_dose"].append(relative_dose_temp)
                        perc_det_summary_thresholds_fixed_reference["recall"].append(perc_detected_temp)
                        perc_det_summary_thresholds_fixed_reference["interpolated"].append(interpolated)

perc_det_summary_thresholds_fixed_reference = pd.DataFrame(perc_det_summary_thresholds_fixed_reference)

In [ ]:
new_names_relative_dose = {0.5: "<b>050%", 0.6: "<b>060%", 0.7: "<b>070%", 0.8: "<b>080%", 0.9: "<b>090%", 1.0: "<b>100%", 0.0: "<b>000%", 0.1: "<b>010%", 0.2: "<b>020%", 0.3: "<b>030%", 0.4: "<b>040%"}
perc_det_summary_thresholds["relative_dose"] = perc_det_summary_thresholds["relative_dose"].map(new_names_relative_dose)
perc_det_summary_thresholds_fixed_reference["relative_dose"] = perc_det_summary_thresholds_fixed_reference["relative_dose"].map(new_names_relative_dose)

In [ ]:
# import plotly.express as px

# df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall_min_dose.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, True, True, False])


# fig = px.strip(df_lesion_meta_recall_prop_rep_max_recall_min_dose[df_lesion_meta_recall_prop_rep_max_recall_min_dose["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"]) & 
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["network"].isin(["<b>3D", "<b>3Dres"])],
#          x="lesion_prop", y="recall", color="reco+kernel", facet_col="network", facet_row="ct", range_y=[0, 1.05], template="simple_white",
#          color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#          category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},
#          stripmode="group"
         
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# #fig.update_layout(title="<b>% Detected lesions at dose levels relative to max detection dose")
# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1)

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.update_layout(
#     height=600,
#     width=700,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_max_lesion_prop.png")


In [ ]:
# df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall_min_dose.\
#     sort_values(["ct", "network" , "R in mm", "C in HU"], ascending=[True, True, False, True])

# fig = px.strip(df_lesion_meta_recall_prop_rep_max_recall_min_dose[~df_lesion_meta_recall_prop_rep_max_recall_min_dose["lesion_prop"].str.endswith("-20HU") & 
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["network"].isin(["<b>3D", "<b>3Dres"]) &
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])],
#          x="lesion_prop", y="recall", color="reco+kernel", facet_col="network", facet_row="ct", range_y=[0, 1.05], template="simple_white",
#          color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#          category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},
#          stripmode="group"
         
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# #fig.update_layout(title="<b>% Detected lesions at dose levels relative to max detection dose")
# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1)

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# fig.update_layout(
#     height=600,
#     width=700,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_max_lesion_prop_filtered.png")


In [ ]:
# df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall_min_dose.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, True, True, False])

# fig = px.bar(df_lesion_meta_recall_prop_rep_max_recall_min_dose[#~df_lesion_meta_recall_prop_rep_max_recall_min_dose["lesion_prop"].str.endswith("-20HU") &
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["network"].isin(["<b>3D", "<b>3Dres"]) &
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["rep"].isin([1]) & 
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])],
#          x="lesion_prop", y="CTDIvol", color="reco+kernel", facet_col="network", facet_row="ct", template="simple_white",
#          color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], barmode = "group",
#          category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],}, hover_data=["CTDIvol"], 
#          range_y=[0, max(df_lesion_meta_recall_prop_rep["CTDIvol"])+0.1]

# )

# #fig.update_layout(title="<b>% Detected lesions at dose levels relative to max detection dose")
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_yaxes(title_text="<b>CTDIvol", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# #fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))


# fig.update_layout(
#     height=600,
#     width=700,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )
# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_max_lesion_prop_dose.png")


In [ ]:
# df_lesion_meta_recall_prop_rep_max_recall_min_dose = df_lesion_meta_recall_prop_rep_max_recall_min_dose.\
#     sort_values(["ct", "network" , "R in mm", "C in HU"], ascending=[True, True, False, True])

# fig = px.bar(df_lesion_meta_recall_prop_rep_max_recall_min_dose[~df_lesion_meta_recall_prop_rep_max_recall_min_dose["lesion_prop"].str.endswith("-20HU") &
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["network"].isin(["<b>3D", "<b>3Dres"]) & 
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["rep"].isin([1]) &
#                                                                   df_lesion_meta_recall_prop_rep_max_recall_min_dose["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])],
#          x="lesion_prop", y="CTDIvol", color="reco+kernel", facet_col="network", facet_row="ct", template="simple_white",
#          color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#          category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],}, hover_data=["CTDIvol"],
#          range_y=[0, max(df_lesion_meta_recall_prop_rep["CTDIvol"])+0.1], barmode="group"

# )

# #fig.update_layout(title="<b>% Detected lesions at dose levels relative to max detection dose")
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_yaxes(title_text="<b>CTDIvol", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# #fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))


# fig.update_layout(
#     height=600,
#     width=700,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )
# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_max_lesion_prop_dose_filtered.png")


In [ ]:
#df_lesion_meta_recall_prop_rep_max_recall_min_dose[df_lesion_meta_recall_prop_rep_max_recall_min_dose["rep"].isin([1])].to_excel("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_max_lesion_prop_dose.xlsx", index=False)

In [ ]:
perc_det_summary_thresholds = perc_det_summary_thresholds.sort_values(["relative_dose", "C in HU", "R in mm"], ascending=[False, True, False])

In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, True, True, False])

# fig = px.strip(perc_det_summary_thresholds[(perc_det_summary_thresholds["ct"] == "<b>Vendor 1") & (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="lesion_prop", y="recall", color="reco+kernel", facet_col="relative_dose", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], },
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon.png")


In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, False, True, False])

# fig = px.strip(perc_det_summary_thresholds[(perc_det_summary_thresholds["ct"] == "<b>Vendor 1") & (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="relative_dose", y="recall", color="reco+kernel", facet_col="lesion_prop", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"] },
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon_2.png")


In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, False, True, False])

# fig = px.strip(perc_det_summary_thresholds[perc_det_summary_thresholds["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"]) &
#                                            perc_det_summary_thresholds["reco+kernel"].isin(["HIR"]) &
#                                            perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"])],
#                 x="relative_dose", y="recall", color="ct", facet_col="lesion_prop", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"] },
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon_vs_ge_2.png")


In [ ]:
# perc_det_summary_thresholds_fixed_reference = perc_det_summary_thresholds_fixed_reference.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, False, True, False])

# fig = px.strip(perc_det_summary_thresholds_fixed_reference[(perc_det_summary_thresholds_fixed_reference["ct"] == "<b>Vendor 1") &
#                                                             (perc_det_summary_thresholds_fixed_reference["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="relative_dose", y="recall", color="reco+kernel", facet_col="lesion_prop", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
#                                  "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
#                                   "relative_dose": ["<b>100%", "<b>090%", "<b>080%", "<b>070%", "<b>060%", "<b>050%", "<b>040%", "<b>030%", "<b>020%", "<b>010%", "<b>000%"]},
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon_fixed_ref.png")


In [ ]:
# perc_det_summary_thresholds_fixed_reference = perc_det_summary_thresholds_fixed_reference.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, False, True, False])

# fig = px.strip(perc_det_summary_thresholds_fixed_reference[(perc_det_summary_thresholds_fixed_reference["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])) &
#                                                            perc_det_summary_thresholds_fixed_reference["reco+kernel"].isin(["HIR"]) &
#                                                             (perc_det_summary_thresholds_fixed_reference["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="relative_dose", y="recall", color="ct", facet_col="lesion_prop", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
#                                  "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
#                                   "relative_dose": ["<b>100%", "<b>090%", "<b>080%", "<b>070%", "<b>060%", "<b>050%", "<b>040%", "<b>030%", "<b>020%", "<b>010%", "<b>000%"]},
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon_vs_ge_fixed_ref.png")


In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" , "R in mm", "C in HU"], ascending=[True, True, False, True])

# fig = px.strip(perc_det_summary_thresholds[(~perc_det_summary_thresholds["lesion_prop"].str.endswith("-20HU")) & 
#                                            (perc_det_summary_thresholds["ct"] == "<b>Vendor 1") &
#                                            (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                 x="lesion_prop", y="recall", color="reco+kernel", facet_col="relative_dose", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"] },
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_canon_filtered.png")


In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, True, True, False])

# fig = px.strip(perc_det_summary_thresholds[(perc_det_summary_thresholds["ct"] == "<b>Vendor 2") &
#                                            (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="lesion_prop", y="recall", color="reco+kernel", facet_col="relative_dose", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],}, hover_data=["CTDIvol"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_ge.png")

In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" ,"C in HU", "R in mm"], ascending=[True, False, True, False])

# fig = px.strip(perc_det_summary_thresholds[(perc_det_summary_thresholds["ct"] == "<b>Vendor 2") & (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="relative_dose", y="recall", color="reco+kernel", facet_col="lesion_prop", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"] },
#                 hover_data=["CTDIvol", "interpolated"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()

# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_ge_2.png")


In [ ]:
# perc_det_summary_thresholds = perc_det_summary_thresholds.\
#     sort_values(["ct", "network" , "R in mm", "C in HU"], ascending=[True, True, False, True])

# fig = px.strip(perc_det_summary_thresholds[(~perc_det_summary_thresholds["lesion_prop"].str.endswith("-20HU")) & (perc_det_summary_thresholds["ct"] == "<b>Vendor 2") &
#                                            (perc_det_summary_thresholds["network"].isin(["<b>3D", "<b>3Dres"]))],
#                x="lesion_prop", y="recall", color="reco+kernel", facet_col="relative_dose", facet_row="network", range_y=[0, 1.1], template="simple_white",
#                 color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
#                 category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],}, hover_data=["CTDIvol"]
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>Lesion properties (mm/HU)", row=1, col=3)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
# fig.update_layout(
#     height=600,
#     width=1600,
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
# )

# fig.show()
# fig.write_image("plots/3) deep learning performance eval/peak_performance/lesion_seg_recall_thresholds_ge_filtered.png")

In [ ]:
# df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["lesion_prop"] == "12mm/-40HU"]
# df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"] == "<b>3Dres"]
# df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["ct"] == "<b>Vendor 1"] 
# df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

# fig = px.strip(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
#          facet_col="network", facet_row="reco+kernel", template="simple_white", 
#          range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], stripmode="overlay",
#          category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],},
#          hover_data=["recall_min_stable"])

# fig.update_layout(
#     height=800,
# )

# fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

# fig.update_yaxes(title_text="<b>Recall", col=1)
# fig.update_xaxes(title_text="<b>CTDIvol", row=1)

# fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
# fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))


# fig.show()

In [ ]:
#df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["lesion_prop"] == "08mm/-40HU"]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["ct"] == "<b>Vendor 1"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=["recall_min_stable"])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_canon_dicethreshold_{dice_detection_threshold}.png"), width=1500, height=500)


In [ ]:
df_lesion_meta_recall_prop_rep_lesions_sub = lesion_seg_summary_prop_phantom_rep[lesion_seg_summary_prop_phantom_rep["ct"] == "<b>Vendor 1"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["phantom"] == 241]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=[])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_canon_dicethreshold_{dice_detection_threshold}_phantom_241.png"), width=1500, height=500)


In [ ]:
df_lesion_meta_recall_prop_rep_lesions_sub = lesion_seg_summary_prop_phantom_rep[lesion_seg_summary_prop_phantom_rep["ct"] == "<b>Vendor 1"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["phantom"] == 247]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=[])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_canon_dicethreshold_{dice_detection_threshold}_phantom_247.png"), width=1500, height=500)


In [ ]:
df_lesion_meta_recall_prop_rep_data = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_data = df_lesion_meta_recall_prop_rep_data[df_lesion_meta_recall_prop_rep_data["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]
# rm duplicate rows
#df_lesion_meta_recall_prop_rep_data = df_lesion_meta_recall_prop_rep_data.drop_duplicates(subset=["lesion_prop", "reco+kernel", "CTDIvol", "recall_min_stable", "recall_min_rep", "rep"])
df_lesion_meta_recall_prop_rep_data = df_lesion_meta_recall_prop_rep_data.sort_values(["ct", "network", "CTDIvol", "reco+kernel", "lesion_prop", "rep"])
columns_to_keep = ["ct", "network", "reco+kernel", "lesion_prop", "CTDIvol", "recall_min_rep", "recall", "rep"]
df_lesion_meta_recall_prop_rep_data = df_lesion_meta_recall_prop_rep_data[columns_to_keep]
df_lesion_meta_recall_prop_rep_data.to_csv(os.path.join(output_dir, f"lesion_detection_recall_dose_canon_dicethreshold_{dice_detection_threshold}.csv"), index=False)
df_lesion_meta_recall_prop_rep_data.to_csv(f"/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/promotion/daten_philipp/Figure 3/lesion_detection_recall_dose_dicethreshold_{dice_detection_threshold}.csv", index=False)


In [ ]:
df_lesion_meta_recall_prop_rep_data["network"].unique()

In [ ]:
#df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["lesion_prop"] == "08mm/-40HU"]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["ct"] == "<b>Vendor 2"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=["recall_min_stable"])

fig.update_layout(
    height=500,
    width=1500,
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=18, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_ge_dicethreshold_{dice_detection_threshold}.png"), width=1500, height=500)


In [ ]:
df_lesion_meta_recall_prop_rep_lesions_sub = lesion_seg_summary_prop_phantom_rep[lesion_seg_summary_prop_phantom_rep["ct"] == "<b>Vendor 2"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["phantom"] == 241]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=[])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_ge_dicethreshold_{dice_detection_threshold}_phantom_241.png"), width=1500, height=500)


In [ ]:
df_lesion_meta_recall_prop_rep_lesions_sub = lesion_seg_summary_prop_phantom_rep[lesion_seg_summary_prop_phantom_rep["ct"] == "<b>Vendor 2"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["phantom"] == 247]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="reco+kernel",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white",
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"], 
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"]},
         hover_data=[])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_ge_dicethreshold_{dice_detection_threshold}_phantom_247.png"), width=1500, height=500)


In [ ]:
#df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["lesion_prop"] == "08mm/-40HU"]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D", "<b>3Dres"])]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == 1]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["reco+kernel"] == "HIR"]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)


fig = px.line(df_lesion_meta_recall_prop_rep_lesions_sub, x="CTDIvol", y="recall_min_rep", color="ct",
         facet_row="network", facet_col= "lesion_prop",  template="simple_white", 
         range_y=[0, 1.1], color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
         category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"], 
                          "lesion_prop": ["12mm/-40HU", "12mm/-30HU", "08mm/-40HU", "08mm/-30HU", "12mm/-20HU", "08mm/-40HU"],
                          "network": ["<b>3D", "<b>3Dres"],
                          "CTDIvol": CTDIvol},
         hover_data=["recall_min_stable"])

fig.update_layout(
    height=500,
    width=1500
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))


#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=8, x1=32, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)


fig.show()
fig.write_image(os.path.join(output_dir, f"lesion_seg_recall_dose_canon_vs_ge_dicethreshold_{dice_detection_threshold}.png"))
